In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from PIL import Image
import os
import numpy as np
import cv2

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
# Hyperparameters
batch_size = 32
learning_rate = 0.001
num_epochs = 10

# Data paths and colors
data_path = "/content/drive/MyDrive/Color_Recognition_Model/training_dataset_colors"
colors = ['blue', 'green', 'red', 'yellow', 'black', 'white', 'brown', 'green', 'grey', 'orange', 'violet']

# Custom dataset
class ColorDataset(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = self.data[idx]
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

# Transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((256, 256)),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize to mean=0.5 and std=0.5
])

In [14]:
data = []
labels = []
for color in colors:
    color_path = os.path.join(data_path, color)
    for img_name in os.listdir(color_path):
        img = cv2.imread(os.path.join(color_path, img_name))
        img = cv2.resize(img, (256, 256))  # Resize images to a consistent size
        img_array = img / 255.0  # Normalize pixel values to [0, 1]
        data.append(img_array)
        labels.append(colors.index(color))

In [15]:
# Split data into training and testing sets
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.2, random_state=42)

# Create dataset and dataloaders
train_dataset = ColorDataset(train_data, train_labels, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = ColorDataset(test_data, test_labels, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [16]:

# Define the neural network
class ColorNet(nn.Module):
    def __init__(self, num_classes):
        super(ColorNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 64 * 64, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 64 * 64)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize the model, loss function, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ColorNet(num_classes=len(colors)).to(device, dtype=torch.double)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")


/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Epoch [1/10], Loss: 0.6988
Epoch [2/10], Loss: 0.3477
Epoch [3/10], Loss: 0.0457
Epoch [4/10], Loss: 0.1389
Epoch [5/10], Loss: 0.0558
Epoch [6/10], Loss: 0.0717
Epoch [7/10], Loss: 0.0400
Epoch [8/10], Loss: 0.1072
Epoch [9/10], Loss: 0.4151
Epoch [10/10], Loss: 0.0844


In [ ]:
save_path = "/content/drive/MyDrive/Color_Recognition_Model/Color_Recognition.pth"  # Update with the desired save path
torch.save(model.state_dict(), save_path)

print(f"Model saved to {save_path}")

Model saved to /content/drive/MyDrive/Color_Recognition_Model/Color_Recognition.pth


In [3]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Set the model to evaluation mode
model.eval()

# Lists to store true labels and predicted labels
true_labels = []
predicted_labels = []

# Iterate through the test dataset
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        # Make predictions
        outputs = loaded_model(images)
        _, predicted = torch.max(outputs.data, 1)

        # Append true and predicted labels
        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(predicted.cpu().numpy())

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Calculate confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)
print("Confusion Matrix:")
print(conf_matrix)

# Classification Report
target_names = colors  # List of color names
class_report = classification_report(true_labels, predicted_labels, target_names=target_names)
print("Classification Report:")
print(class_report)


/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Accuracy: 89.09%
Confusion Matrix:
[[ 5  1  0  0  0  0  0  0  0  0]
 [ 0 10  0  0  0  0  0  0  0  0]
 [ 0  0  3  0  0  0  0  0  0  1]
 [ 0  0  0  3  0  0  0  0  1  0]
 [ 0  0  0  0  5  0  0  0  0  0]
 [ 0  0  0  0  0  7  0  0  0  0]
 [ 0  0  0  0  0  0  3  0  0  0]
 [ 0  0  0  0  0  0  0  6  0  0]
 [ 0  0  1  0  0  0  0  0  4  0]
 [ 2  0  0  0  0  0  0  0  0  3]]
Classification Report:
              precision    recall  f1-score   support

        blue       0.71      0.83      0.77         6
       green       0.91      1.00      0.95        10
         red       0.75      0.75      0.75         4
      yellow       1.00      0.75      0.86         4
       black       1.00      1.00      1.00         5
       white       1.00      1.00      1.00         7
       brown       1.00      1.00      1.00         3
       green       0.00      0.00      0.00         0
        grey       1.00      1.00      1.00         6
      orange       0.80      0.80      0.80         5
      violet    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.